In [63]:
from natasha import Doc
from natasha import NewsEmbedding
from natasha import NewsMorphTagger
from natasha import Segmenter
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
from pymystem3 import Mystem
m = Mystem()
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

from string import punctuation
punctuation += '” '

*Я умел и я любим всеми, как мы любим. Раньше он умел печь. Кладу на печь. Тот сундук наносит вред тому кладу. Бог Тот не рад первому тому. Не надо никаких Верховных рад. Надо мною летает стих. Ты почему так резко стих? Где Катя? Катя машину до заправки, я пару раз упал. Машину слезу никто не видел? Сейчас слезу, блин! Довольно вкусный блин. Его эго довольно. Ной не ныл, а ты ной. О, понял! Песня о любви. Но я жал на кнопку. Много пчелиных жал. Жаль, пчела, чтобы было очень жаль! Желаю им совет да любовь! Да! Уже все, не поговорим об уже?*


Почему этот текст окажется сложным для тэггера -- понятно: как видите, тут много омонимов, которые повторяются, но принадлежат к разным лексемам и разным частям речи. "печь", "жаль", "машину", "надо" и т.д. и т.п.

In [8]:
with open("russian_golden.txt", encoding="utf-8") as f:
    russian_golden = f.read()
with open("russian_golden_notes.txt", encoding="utf-8") as f:
    russian_golden_grammemes = f.read().split("\n")

In [9]:
russian_golden_no_punct = russian_golden.translate(str.maketrans('', '', string.punctuation))

In [10]:
mystem_ana = m.analyze(russian_golden)

In [11]:
# граммемы майстема
mystem_grammemes = []
for token in mystem_ana:
    if "analysis" in token:
        gr = token['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        mystem_grammemes.append(pos)

In [35]:
# граммемы пайморфи
pymorphy_grammemes = []
for token in russian_golden_no_punct.split():
    pos = morph.parse(token)[0].tag.POS
    pymorphy_grammemes.append(str(pos))

In [19]:
doc = Doc(russian_golden)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [22]:
# граммемы наташи
natasha_grammemes = []
for token in doc.tokens:
    if token.pos != "PUNCT":
        natasha_grammemes.append(token.pos)

Для того, чтобы можно было приравнять разные грамеммы, я создал словарь русских граммем. Ключи -- то, как я обозначил части речи у себя, значения -- то, как их по-разному могут обозначать парсеры (поскольку нам важна только часть речи, мы пренебрегаем тем, что в списке могут быть две грамеммы с разным смыслом, но обозначающие одну часть речи).

In [76]:
rus_golden_pos_dict = {
    "A": ["A", "ADJ", "ADJF"],
    "ADV": ["ADV", "ADVB"],
    "ANUM": ["ANUM", "A", "ADJ"],
    "ADVPRO": ["ADVPRO", "ADV", "ADVB", "PRON",],
    "APRO": ["APRO", "A", "PRON", "ADJ", "ADJF"],
    "COMP": ["COMP", "ADV"],
    "CONJ": ["CONJ", "CCONJ",],
    "INTJ": ["INTJ",],
    "PART": ["PART", "PRCL"],
    "PR": ["PR", "ADP", "PREP"],
    "S": ["S", "NOUN",],
    "SPRO": ["SPRO", "S", "NOUN", "PRON", "NPRO"],
    "V": ["V", "VERB",],
    "DET": ["DET", "APRO", "A", "ADJ", "ADJF"],
    "INFN": ["V", "VERB", "INFN"],
    "ADJS": ["A", "ADJ", "ADJS"],
    "PRED": ["PRED", "ADV", "ADVB"]
}

Небольшая функция для accuracy. на вход: список граммем, словарь, список "золотых" или правильных граммем (написанных вручную).
На выходе -- доля правильных граммем.

In [223]:
def accuracy(ana, dic, golden):
    correct = 0
    for pos, pos_golden in zip(ana, golden):
        if pos in dic[pos_golden]:
            correct += 1
    return correct/len(golden)

In [224]:
accuracy(pymorphy_grammemes, rus_golden_pos_dict, russian_golden_grammemes)

0.7307692307692307

In [225]:
accuracy(natasha_grammemes, rus_golden_pos_dict, russian_golden_grammemes)

0.7596153846153846

In [226]:
accuracy(mystem_grammemes, rus_golden_pos_dict, russian_golden_grammemes)

0.8076923076923077

Как мы видим, майстем значительно лучше остальных.

Теперь посмотрим на английские парсеры:

In [70]:
import spacy
import nltk
from nltk import word_tokenize

In [207]:
from flair.models import SequenceTagger
from flair.data import Sentence

tagger = SequenceTagger.load('pos')

2020-10-18 22:07:32,581 loading file /Users/fixed/.flair/models/en-pos-ontonotes-v0.5.pt


*The complex houses married and single soldiers and their families. “I see,” said the blind man as he picked up the hammer and saw. The old man the boats. Will Will Smith smith? Do you mind if I change my mind? If you like it, give it a like! Leaves leave every fall. I’m feeling weird about having a feeling. I’m well, but I fell inside a well. I was the first one to coin the term coin. The bully wants to bully me. Love you, my love. I hate my fucking life. Are those guys fucking? I think that they are.*

Похожая ситуация, что и в русском тексте: много омонимов, которые принадлежат разным частям речи: *love*, *feeling*, *leave*, *saw*.

In [228]:
with open("english_golden.txt", encoding="utf-8") as f:
    english_golden = f.read().strip('\n')
with open("english_golden_notes.txt", encoding="utf-8") as f:
    english_golden_grammemes = f.read().strip('\n').split("\n")

In [208]:
sentence = Sentence(english_golden)
tagger.predict(sentence)

In [211]:
# граммемы flair
flair_grammemes = []
for entity in sentence.to_dict(tag_type="pos")['entities']:
    if entity['text'] not in punctuation:
        flair_grammemes.append(str(entity['labels'][0]).split()[0])

In [195]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(english_golden)

In [196]:
# граммемы spacy
spacy_grammemes = []
for token in doc:
    if token.text not in punctuation:
        spacy_grammemes.append(token.pos_)

In [197]:
tokens_nltk = word_tokenize(english_golden)

In [203]:
# граммемы nltk
nltk_grammemes = []
for token in nltk.pos_tag(tokens_nltk):
    if token[0] not in punctuation:
        nltk_grammemes.append(token[1])

Создаем аналогичный словарь для английских граммем:

In [233]:
eng_golden_pos_dict = {
    "DET": ["DT", "DET"],
    "NOUN": ["NOUN", "NN", "NNP", "NNS"],
    "ADJ": ["ADJ", "JJ"],
    "VERB": ["VBP", "VBD", "VBG", "VBZ", "VB", "VERB"],
    "CONJ": ["CC", "CCONJ", "SCONJ", "IN"],
    "PRON": ["PRON", "PRP"],
    "AUX": ["AUX", "VBP", "VBD", "VERB"],
    "NUM": ["CD", "NUM"],
    "PART": ["TO", "PART"],
    "PRP$": ["PRP$", "DET", "DT"],
    "ADP": ["ADP", "RP", "RB", "IN"],
    "ADV": ["ADV"]
}

Функция у нас уже есть:

In [234]:
accuracy(spacy_grammemes, eng_golden_pos_dict, english_golden_grammemes)

0.8653846153846154

In [235]:
accuracy(nltk_grammemes, eng_golden_pos_dict, english_golden_grammemes)

0.875

In [236]:
accuracy(flair_grammemes, eng_golden_pos_dict, english_golden_grammemes)

0.8846153846153846

Мы видим, что в целом accuracy гораздо выше русской: самый низкий английский результат примерно на 6 процентов выше самого высокого русского.

Для последнего пункта я выбрал три конструкции: "совсем не A", "A, хотя A" и "ADV A". Я предполагаю, что вторая и третья чаще встречаются в положительных отзывах ("интересный, хотя длинноватый" -- в положительных отзывах чаще можно встретить, как человек прощает недостаток, интуитивно кажется, что это вероятнее, чем если бы кто-то ругал фильм, но не забыл упомянуть плюс; "удивительно легкий", "очень хороший" кажутся более, а первая -- наоборот. 

In [330]:
def myst_pos(token):
    if 'analysis' in token:
        return token['analysis'][0]['gr'].split('=')[0].split(',')[0]

def phrase_getter(text):
    ana = m.analyze(text)
    newtext = []
    j = 0
    
    while ana:
        
        if myst_pos(ana[0]) == "ADV" and myst_pos(ana[2]) == "A":
            newtext.append([tok['text'] for tok in ana[:3]])
            ana = ana[3:]
        elif ana[0]['text'] == "совсем" and ana[2]["text"] == "не" and myst_pos(ana[4]) == "A":
            newtext.append([tok['text'] for tok in ana[:5]])
            ana = ana[5:]
        elif myst_pos(ana[0]) == "A" and ana[2]['text'] == 'хотя' and myst_pos(ana[4]) == "A":
            newtext.append([tok['text'] for tok in ana[:5]])
            ana = ana[5:]
        else:
            newtext.append(ana[0]['text'])
            ana = ana[1:]
        
    return newtext

In [332]:
phrase_getter("красивый, хотя длинноватый фильм")

[['красивый', ', ', 'хотя', ' ', 'длинноватый'], ' ', 'фильм', '\n']

In [334]:
phrase_getter("очень хорошие актеры")

[['очень', ' ', 'хорошие'], ' ', 'актеры', '\n']

In [335]:
phrase_getter("на совсем не релеватные темы")

['на', ' ', ['совсем', ' ', 'не', ' ', 'релеватные'], ' ', 'темы', '\n']

Посмотреть, насколько хорошо это добавление влияет на первую домашку, я не успел :(

*Миша Сонькин*